# Import Libraries

In [1]:
import os
from langchain_openai import OpenAIEmbeddings
import hashlib
from pinecone import Pinecone
from datetime import date

c:\Codes\Python\notebooks\echo_chatbot\.chatbot\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


# API Keys

In [3]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

# Initialization

In [5]:
index = input("Input organization name")
namespace = input("Input meeting title")

# Pinecone Initialization
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(index)

# OpenAI Initialization
EMBEDDINGS = OpenAIEmbeddings(model='text-embedding-3-small', openai_api_key=OPENAI_API_KEY)

### Sample Transcript

In [10]:
sample_transcript = """Czech: Hello my name is Czech.     
Gian: Hello my name is Gian.
Shaundyl: Hello my name is Shaundyl.
Czech (Team Lead): Alright, everyone, thanks for joining today’s meeting. We have about 10 minutes to go over the final details before the product launch. Let's start with the progress update. Bob, how are we doing on the development front?
Shaundyl (Developer): Things are looking good. We’ve implemented all the major features. I’m currently working on the final round of bug fixes. I should be done with it by tomorrow, but I need the QA team to give it another pass afterward.
Czech: Great to hear that. Gian, how are we looking on the project timeline? Any changes or concerns?
Gian (Project Manager): We’re on track, but barely. The marketing materials were delayed by two days, but I managed to align the social media schedule to compensate for the delay. As long as the development and testing stay on track, we should meet the launch date. We might want to allocate some buffer time for any last-minute issues though.
Czech: Makes sense. Shaundyl, do you think we’ll have time for a final round of testing before we push live?
Shaundyl: We should. I’ll aim to finish bug fixes by midday tomorrow. If QA can start immediately after, we’ll have 24 hours for testing before the go-live. I’ll stay available for any hotfixes, just in case.
Czech: Perfect. Gian, can you make sure the QA team is on standby for tomorrow afternoon?
Gian: Absolutely. I’ll notify them as soon as we’re done here. I’ll also double-check the launch checklist to make sure nothing’s been missed.
Czech: Great. And one last thing – how are we handling customer support on launch day? Any special preparations?
Gian: We’ve set up a dedicated support channel for the product and briefed the customer support team on the common issues we’re anticipating. We’ll also monitor social media for any unexpected feedback.
Czech: Sounds like we’re in good shape. Thanks, everyone. Let’s aim to regroup tomorrow for a final status check. Anything else before we wrap up?
Shaundyl: Nothing from my side. I’ll update you if any blockers come up.
Gian: I’m all set. Let’s get this done!
Czech: Alright then, thanks again! Talk tomorrow.
"""

# Chunking

In [11]:
def chunk_text(text, max_chunk_size=500):
    # Ensure each text ends with a newline to correctly split sentences
    if not text.endswith("\n"):
        text += "\n"

    # Split text into sentence
    sentences = text.split("\n")
    chunks = []
    current_chunk = ""

    # Iterate over sentence and assemble chunks
    for sentence in sentences:
        # Check if adding the current sentence exceeds the maximum chunk size
        if (len(current_chunk) + len(sentences) + 2 > max_chunk_size and current_chunk):
            # Add the current chunk to the list and start a new chunk
            chunks.append(current_chunk.strip())
            current_chunk = ""
        # Add the current sentence to the current chunk
        current_chunk += sentence.strip() + "\n"
    # Add any remaining text as the last chunk
    if (current_chunk):
        chunks.append(current_chunk.strip())

    return chunks # type: list[str]

chunked_text = chunk_text(text=sample_transcript)
print(chunked_text)

["Czech: Hello my name is Czech.\nGian: Hello my name is Gian.\nShaundyl: Hello my name is Shaundyl.\nCzech (Team Lead): Alright, everyone, thanks for joining today’s meeting. We have about 10 minutes to go over the final details before the product launch. Let's start with the progress update. Bob, how are we doing on the development front?\nShaundyl (Developer): Things are looking good. We’ve implemented all the major features. I’m currently working on the final round of bug fixes. I should be done with it by tomorrow, but I need the QA team to give it another pass afterward.", 'Czech: Great to hear that. Gian, how are we looking on the project timeline? Any changes or concerns?\nGian (Project Manager): We’re on track, but barely. The marketing materials were delayed by two days, but I managed to align the social media schedule to compensate for the delay. As long as the development and testing stay on track, we should meet the launch date. We might want to allocate some buffer time f

# Generate Embeddings

In [ ]:
def generate_embeddings(texts):
    """
    Generate embeddings for a list of text.
    """
    embedded = EMBEDDINGS.embed_documents(texts)

    print("Generating embeddings: Done!")
    return embedded

chunked_text_embeddings = generate_embeddings(texts=chunked_text)
print(chunked_text_embeddings)

# Combine Vector and Text

In [ ]:
def generate_short_id(content):
    """
    Generate a short ID based on the content using SHA-256 hash.
    """
    hash_obj = hashlib.sha256()
    hash_obj.update(content.encode("utf-8"))

    print("Generating short id: Done!")
    return hash_obj.hexdigest()

def combine_vector_and_text(texts, meeting_title, date, text_embeddings):
    """
    Process a list of texts along with their embeddings.
    """
    data_with_metadata = []

    for doc_text, embedding in zip(texts, text_embeddings):
        if not isinstance(doc_text, str):
            doc_text = str(doc_text)

        if not isinstance(meeting_title, str):
            meeting_title = str(meeting_title)

        if not isinstance(date, str):
            date = str(date)

        text_id = generate_short_id(doc_text)
        data_item = {
            "id": text_id,
            "values": embedding,
            "metadata": {"text": doc_text, "title": meeting_title, "date": date},
        }

        data_with_metadata.append(data_item)

    print("Combining vector and text: Done!")
    return data_with_metadata

data_with_meta_data = combine_vector_and_text(texts=chunked_text, meeting_title=namespace, date=date, text_embeddings=chunked_text_embeddings)
print(data_with_meta_data)

# Upsert to Pinecone

In [ ]:
def upsert_data_to_pinecone(data_with_metadata, namespace):
    """
    Upsert data with metadata into a Pinecone index.
    """
    index.upsert(vectors=data_with_metadata, namespace=namespace)
    print("Upserting vectors to Pinecone: Done!")

upsert_data_to_pinecone(data_with_metadata=data_with_meta_data, namespace=namespace)
index.describe_index_stats()

# MAIN

In [ ]:
def Pinecone(texts, meeting_title):
    today = str(date.today()) # INITIALIZATION FOR DATE (DYNAMIC) BASED ON STORING
    namespace = 'USJ-R' # NAMESPACE DEFAULTED TO 'USJ-R' FOR ISOLATION (STATIC)

    chunked_text = chunk_text(text=texts)
    chunked_text_embeddings = generate_embeddings(texts=chunked_text)
    data_with_meta_data = combine_vector_and_text(texts=chunked_text, meeting_title=meeting_title, date=today,  text_embeddings=chunked_text_embeddings)
    upsert_data_to_pinecone(data_with_metadata=data_with_meta_data, namespace_name=namespace)

Pinecone(texts=text1, meeting_title="Kickoff Meeting for Software Development Project")